In [1]:
import os
import random
from tifffile import imwrite, imread
from readlif.reader import LifFile
import napari
import numpy as np
from dask import delayed
import dask.array as da

from basicpy import BaSiC

In [2]:
# where to save tiled images
im_dir = r'R:\Betsy\Thunder_Ovcar\240422_OC_analysis\tiled_images'

save_dir = r'R:\Betsy\Thunder_Ovcar\240422_OC_analysis\tiled_images_basic' 
save_basic_dir = r'R:\Betsy\Thunder_Ovcar\240422_OC_analysis\basic_fields'

In [3]:
# define wells to be analyzed
well_list = list(set([x.split('_')[0] for x in os.listdir(im_dir) if 'tif' in x]))
print(len(well_list))
well_list

20


['E4',
 'B8',
 'D3',
 'A1',
 'A5',
 'H7',
 'F11',
 'E5',
 'H8',
 'G10',
 'F9',
 'D9',
 'A4',
 'G2',
 'B7',
 'C10',
 'H3',
 'H1',
 'D2',
 'B11']

## Correct illumination with Basic

In [4]:
round_list = ['00','01']


for r in round_list:

    channel_list  = set([f[f.index('ch_')+3:][:2] for f in os.listdir(im_dir) if (f'r_{r}' in f)])

    for ch in channel_list:

        im_files = [os.path.join(im_dir, f) for f in os.listdir(im_dir) if (f'r_{r}_ch_{ch}' in f)]

        # read in images
        im_list = []
        for im_name in im_files:
            im = imread(im_name)
            im_list.append(im)

        # select signal images
        im_sel = []
        name_sel = []
        for im,im_name in zip(im_list,im_files):
            if np.max(im)>0:
                im_sel.append(im)
                name_sel.append(im_name)
            else:
                pass
                #imwrite(os.path.join(save_dir, os.path.basename(im_name)), im.astype('uint16'))
        
        im_test = np.array(im_sel)

        print('Ready for basic')

        # run Basic
        basic = BaSiC(get_darkfield=False, smoothness_flatfield=1, smoothness_darkfield=1)
        basic.fit(np.array(random.sample(im_sel,50)))

        images_transformed = basic.transform(im_test)

        # save basic fields
        imwrite(os.path.join(save_basic_dir, f'r_{r}_ch_{ch}_flatfield_basic.tif'), basic.flatfield)
        imwrite(os.path.join(save_basic_dir, f'r_{r}_ch_{ch}_darkfield_basic.tif'), basic.darkfield)

        # save tiles
        for im,im_name in zip(images_transformed,name_sel):
            imwrite(os.path.join(save_dir, os.path.basename(im_name)), im.astype('uint16'))


Ready for basic


KeyboardInterrupt: 